In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE 
import local_python_packages.features_adding as local

In [40]:
pd.set_option('display.max_columns', None)
flights_data = pd.read_csv('All carrier flights 2019.csv',parse_dates=[0])
flights_data = flights_data.sort_values(['fl_date'])

In [41]:
df_weather = pd.read_csv('cities_and_dates_weather_final.csv',parse_dates=[1])

In [42]:
df_holidays = pd.read_csv('US holidays.csv',parse_dates=[0])

Adding additional columns from our feature engineering

In [43]:
flights_data = local.add_taxi_Ndays_rolling(flights_data, 30)

In [44]:
flights_data = local.add_traffic_rolling(flights_data, 30)

In [45]:
flights_data = local.make_month_dummies(flights_data, 'fl_date')

In [46]:
flights_data = local.merging_weather_flights(flights_data,df_weather)

In [47]:
flights_data = local.add_dep_delay_Ndays_rolling(flights_data, 30)

In [48]:
flights_data = local.add_US_holidays(flights_data, df_holidays)

In [49]:
flights_data = local.make_dates_ordinal(flights_data, 'fl_date')

In [50]:
flights_data['orig_air by date']= flights_data['fl_date'] * flights_data['origin_airport_id'] 

In [51]:
flights_data['dest_air by date']= flights_data['fl_date'] * flights_data['dest_airport_id'] 

In [52]:
flights_data['mkt_carrier_fl_num by date']= flights_data['fl_date'] * flights_data['mkt_carrier_fl_num']  

In [53]:
flights_data['tail_num by date'] = flights_data['fl_date'] * hash(str(flights_data['tail_num']))

In [54]:
flights_data['tail_num by dest airport'] = flights_data['dest_airport_id'] * hash(str(flights_data['tail_num']))

In [55]:
flights_data['tail_num by origin airport'] = flights_data['origin_airport_id'] * hash(str(flights_data['tail_num']))

In [56]:
flights_data['mkt_carrier by dest_airport']= flights_data['mkt_carrier_fl_num']  * flights_data['dest_airport_id'] 

In [57]:
flights_data['mkt_carrier by origin_airport']= flights_data['mkt_carrier_fl_num']  * flights_data['origin_airport_id'] 

In [62]:
features_list = ['fl_date','taxi_out','taxi_in', 'arr_delay',
                 'crs_elapsed_time',
       'air_time', 'distance',
       '30d taxi_out', '30d taxi_in',
       '30d roll flts origin_airport_id', '30d roll flts dest_airport_id',
       #'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 
                 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'origin_city_wspd', 'origin_visibility',
       'dest_city_wspd', 'dest_visibility', 'origin_cond_Overcast',
       'origin_cond_Partially cloudy', 'origin_cond_Rain', 'origin_cond_Snow',
       'dest_cond_Overcast', 'dest_cond_Partially cloudy', 'dest_cond_Rain',
       'dest_cond_Snow','30 days roll dep_time', 'Type_Federal holiday', 'dest_air by date' , 'orig_air by date',
                'origin_wspd/visib', 'dest_wspd/visib', 'mkt_carrier_fl_num by date', 'tail_num by date',
                 'tail_num by dest airport', 'tail_num by origin airport', 'mkt_carrier by dest_airport', 
                 'mkt_carrier by origin_airport']

In [63]:
flights_data = local.replace_nan_with_mean(flights_data, 'arr_delay')

In [64]:
flights_data.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,30d taxi_out,30d taxi_in,30d roll flts origin_airport_id,30d roll flts dest_airport_id,month_6,month_7,month_8,month_9,month_10,month_11,month_12,origin_city_wspd,origin_visibility,origin_wspd/visib,dest_city_wspd,dest_visibility,dest_wspd/visib,origin_cond_Overcast,origin_cond_Partially cloudy,origin_cond_Rain,origin_cond_Snow,dest_cond_Overcast,dest_cond_Partially cloudy,dest_cond_Rain,dest_cond_Snow,30 days roll dep_time,Type_Federal holiday,orig_air by date,dest_air by date,mkt_carrier_fl_num by date,tail_num by date,tail_num by dest airport,tail_num by origin airport,mkt_carrier by dest_airport,mkt_carrier by origin_airport
0,737233,AA,5709,N575NN,5709,11057,CLT,"Charlotte, NC",15624,VPS,"Valparaiso, FL",1740,1735.0,-5.0,12.0,1747.0,1759.0,9.0,1818,1808.0,-10.0,0.0,NaN,0.0,N,98.0,93.0,72.0,1.0,460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,13.3,12.8,1.039032,17.5,16.0,1.093691,0,1,0,0,0,0,0,0,NaN,0,8151585281,11518528392,4208863197,2954779933571463579,-5673930564194530216,7583471808232444603,89197416,63124413
1,737233,HA,8,N388HA,8,12173,HNL,"Honolulu, HI",12889,LAS,"Las Vegas, NV",2240,2239.0,-1.0,16.0,2255.0,713.0,10.0,725,723.0,-2.0,0.0,NaN,0.0,N,345.0,344.0,318.0,1.0,2762.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,18.3,16.0,1.143660,15.2,16.0,0.950031,0,0,1,0,0,0,0,0,NaN,0,8974337309,9502196137,5897864,2954779933571463579,920382926508884595,-3362805559901194193,103112,97384
2,737233,HA,9,N388HA,9,12892,LAX,"Los Angeles, CA",12173,HNL,"Honolulu, HI",1810,1806.0,-4.0,14.0,1820.0,2037.0,7.0,2100,2044.0,-16.0,0.0,NaN,0.0,N,350.0,338.0,317.0,1.0,2556.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,16.6,15.3,1.084912,18.3,16.0,1.143660,0,1,0,0,0,0,1,0,NaN,0,9504407836,8974337309,6635097,2954779933571463579,-3362805559901194193,-6249717675594096780,109557,116028
3,737233,HA,10,N388HA,10,12173,HNL,"Honolulu, HI",12892,LAX,"Los Angeles, CA",800,759.0,-1.0,13.0,812.0,1559.0,10.0,1630,1609.0,-21.0,0.0,NaN,0.0,N,330.0,310.0,287.0,1.0,2556.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,18.3,16.0,1.143660,16.6,15.3,1.084912,0,0,1,0,0,1,0,0,NaN,0,8974337309,9504407836,7372330,2954779933571463579,-6249717675594096780,-3362805559901194193,128920,121730
4,737233,HA,11,N390HA,11,14771,SFO,"San Francisco, CA",12173,HNL,"Honolulu, HI",915,913.0,-2.0,15.0,928.0,1122.0,5.0,1140,1127.0,-13.0,0.0,NaN,0.0,N,325.0,314.0,294.0,1.0,2398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,31.7,14.5,2.185389,18.3,16.0,1.143660,0,0,0,0,0,0,1,0,NaN,0,10889668643,8974337309,8109563,2954779933571463579,-3362805559901194193,3882825859035829649,133903,162481


In [65]:
flights_data = flights_data[features_list]

In [66]:
flights_data = flights_data.dropna()

In [67]:
features_list.remove('arr_delay')

In [68]:
X = flights_data[features_list]
y = flights_data['arr_delay']

In [69]:
scaler = StandardScaler()

In [70]:
X = scaler.fit_transform(X)

In [71]:
X_train, X_test, y_train, y_test = local.quick_split(X,y,train_ratio=0.75)

In [72]:
# xgb_r = xgb.XGBRegressor(
#     objective='reg:squarederror',
#     n_estimators=3438,
#     random_state=17,    
#     verbosity=1,
#     n_jobs=5,
#     booster='gbtree'
# )    

In [73]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [74]:
params = {
    # Parameters that we are going to tune.
    'max_depth':10,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 0.5,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'mae',
    'booster':'gbtree',
    'n_estimators':3438,
    'random_state':17,
    'verbosity':1,
    'n_jobs':5,
    'random_state':17
}

In [75]:
num_boost_round=999

In [76]:
xgb_r = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=30
)

[04:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	Test-mae:23.23560
[1]	Test-mae:22.65453
[2]	Test-mae:22.39123
[3]	Test-mae:22.41453
[4]	Test-mae:22.43643
[5]	Test-mae:22.46926
[6]	Test-mae:22.38354
[7]	Test-mae:22.20586
[8]	Test-mae:22.11899
[9]	Test-mae:22.07268
[10]	Test-mae:22.01344
[11]	Test-mae:21.97370
[12]	Test-mae:21.92677
[13]	Test-mae:21.82533
[14]	Test-mae:21.76590
[15]	Test-mae:21.54159
[16]	Test-mae:21.52110
[17]	Test-mae:21.49953
[18]	Test-mae:21.40568
[19]	Test-mae:21.39320
[20]	Test-mae:21.37951
[21]	Test-mae:21.37167
[22]	Test-mae:21.35496
[23]	Test-mae:21.33104
[24]	Test-mae:21.19158
[25]	Test-mae:21.16863
[26]	Test-mae:21.15953

In [77]:
y_pred = xgb_r.predict(dtest) 

In [78]:
rmse = np.sqrt(MSE(y_test, y_pred)) 
print("RMSE : % f" %(rmse)) 


RMSE :  48.296523


In [79]:
r2_score(y_test, y_pred)

0.14775989150736013